In [31]:
# #!/usr/bin/env python
# # coding: utf-8
# # 


# ### PARSEO DE ARGUMENTOS

# import argparse

# parser = argparse.ArgumentParser(description='A script to process data for a range of years')

# parser.add_argument('-y','--years', nargs='+', help='Set the range of years to process data for. Default is the current year and the next year', required=False, type=int, default=[2022, 2023])
# parser.add_argument('-ow','--overwrite', nargs=1, required=False, default= True, help='Flag to specify if previous data should be overwritten. Default is True')

# args = parser.parse_args()

# overwrite = args.overwrite
# startyr = args.years[0]
# endyr = args.years[1]

In [32]:
overwrite = 1
startyr = 2024
endyr = 2025
name = "202507" # "ARG"

In [33]:



# Column names
y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
          'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
          'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']

x_cols1 = ['IX_TOT', 'P02', 'P03', 'AGLO_rk', 'Reg_rk', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']

x_cols2 = x_cols1 + predecir1
predecir2 = ['INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']

x_cols3 = x_cols2 + predecir2
predecir3 = ['PP07G1','PP07G_59', 'PP07I', 'PP07J', 'PP07K']

# Columnas de ingresos. Necesitan una regresion...
columnas_pesos = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

x_cols4 = x_cols3 + predecir3

predecir4 = columnas_pesos
y_cols4 = predecir4


In [34]:

# #####  TRAINING  #####

# #####  Funciones clasificador y regresor  #####

# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# import os
# import joblib

# def fit_model(train_data, x_cols, y_cols, out_filename,
#              model):
#     X = train_data[x_cols]
#     y = train_data[y_cols]
    
#     X, X_test, y, y_test = train_test_split(X, y, test_size=0.1) # less memory used
    
#     clf = model.fit(X.values, y.values) # fit model

#     # Detect task type based on model class
#     if isinstance(model, RandomForestClassifier):
#         task = 'classification'
#     elif isinstance(model, RandomForestRegressor):
#         task = 'regression'
#     else:
#         task = 'unknown'

#     # Evaluate if supported
#     if task in ['classification', 'regression']:
#         print(f"Evaluating model: {out_filename}")
#         evaluate_model(clf, X_test, y_test, task=task)
#     else:
#         print(f"⚠️ Unknown model type for evaluation: {type(model)}")

#     # save the model to disk
#     if not os.path.exists('./modelos/'):
#         os.makedirs('./modelos/')
#     joblib.dump(model, out_filename, compress=3)
#     print('saved model at: ' + out_filename)

# #     print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])
#     del clf
#     del X; del y # liberar memoria eliminando los dataframes mas pesados


# from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
# from sklearn.multiclass import OneVsRestClassifier

# def evaluate_model(model, X_test, y_test, task='classification'):
#     y_pred = model.predict(X_test)
    
#     if task == 'classification':
#         print("📊 Classification Report:")
#         print(classification_report(y_test, y_pred, zero_division=0))
#         print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#     elif task == 'regression':
#         print("📈 Regression Metrics:")
#         print("MSE:", mean_squared_error(y_test, y_pred))
#         print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
#         print("R²:", r2_score(y_test, y_pred))


In [35]:
import os
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix,
    mean_squared_error, r2_score
)


def evaluate_model(model, X_test, y_test, task='classification', target_names=None, target_col=None):
    y_pred = model.predict(X_test)

    if task == 'classification':
        print(f"\n📊 Report for target: {target_col or 'Unnamed'}")
        print(classification_report(y_test, y_pred, zero_division=0, target_names=target_names))
        print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    elif task == 'regression':
        print(f"\n📈 Regression Report for target: {target_col or 'Unnamed'}")
        print("MSE:", mean_squared_error(y_test, y_pred))
        print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
        print("R²:", r2_score(y_test, y_pred))



# def evaluate_model(model, X_test, y_test, task='classification', target_names=None):
#     y_pred = model.predict(X_test)

#     if task == 'classification':
#         if y_test.ndim == 1 or isinstance(y_test, pd.Series):
#             print("📊 Classification Report:")
#             print(classification_report(y_test, y_pred, zero_division=0, target_names=target_names))
#             print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
#         else:
#             # Multi-output: iterate over columns
#             for i, col in enumerate(y_test.columns):
#                 print(f"\n📊 Report for target: {col}")
#                 print(classification_report(y_test[col], y_pred[:, i], zero_division=0))
#                 print("Confusion Matrix:\n", confusion_matrix(y_test[col], y_pred[:, i]))

#     elif task == 'regression':
#         if y_test.ndim == 1 or isinstance(y_test, pd.Series):
#             y_pred_flat = y_pred
#             y_test_flat = y_test
#         else:
#             # average across columns (summary metric only)
#             y_pred_flat = y_pred.ravel()
#             y_test_flat = y_test.values.ravel()

#         print("📈 Regression Metrics:")
#         print("MSE:", mean_squared_error(y_test_flat, y_pred_flat))
#         print("RMSE:", mean_squared_error(y_test_flat, y_pred_flat, squared=False))
#         print("R²:", r2_score(y_test_flat, y_pred_flat))


def fit_model(train_data, x_cols, y_cols, out_filename, model):
    X = train_data[x_cols]
    y = train_data[y_cols]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    clf = model.fit(X_train.values, y_train.values)

    # Detect task type
    if isinstance(model, RandomForestClassifier):
        task = 'classification'
    if isinstance(model, HistGradientBoostingClassifier):
        task = 'classification'
    elif isinstance(model, RandomForestRegressor):
        task = 'regression'
    else:
        task = 'unknown'

    if task != 'unknown':
        print(f"\n📂 Evaluating: {out_filename}")
        evaluate_model(clf, X_test, y_test, task=task)
    else:
        print(f"⚠️ Unsupported model type: {type(model)}")

    # Save model
    os.makedirs('./modelos/', exist_ok=True)
    joblib.dump(model, out_filename, compress=3)
    print(f"✅ Model saved at: {out_filename}\n")

    # Cleanup
    del clf, X, y, X_train, y_train, X_test, y_test


In [36]:
# pip install xgboost

In [39]:
    
##########################################################################################    
################ Loop principal. Entrenar y guardar modelos  #############################
##########################################################################################

import pandas as pd
from numpy import log10

from xgboost import XGBClassifier

# Reusable model configuration
def make_model():
    return HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_iter=300,
        max_leaf_nodes=64,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        class_weight="balanced",
        random_state=42,
        verbose=0
    )


# Unified stage runner
def run_stage(train_data, x_cols, y_cols, stage_name):
    for target_col in y_cols:
        out = f'./modelos/{stage_name}_{yr}_{name}_{target_col}'
        if not os.path.exists(out) or overwrite:
            model = make_model()
            fit_model(
                train_data,
                x_cols=x_cols,
                y_cols=[target_col],
                out_filename=out,
                model=model
            )

            
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    try:
        training_data = f'./../../data/training/EPHARG_train_{yr[2:]}.csv'
        train_data = pd.read_csv(training_data)
    except FileNotFoundError:
        print(f"File {training_data} not found for year {yr}, skipping.")
        continue
    

    # Run each stage
    run_stage(train_data, x_cols1, predecir1, "clf1")
    run_stage(train_data, x_cols2, predecir2, "clf2")
    run_stage(train_data, x_cols3, predecir3, "clf3")

2024


/home/matias/anaconda3/envs/new_env/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



📂 Evaluating: ./modelos/clf1_2024_202507_CAT_OCUP


/home/matias/anaconda3/envs/new_env/lib/python3.11/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(



📊 Report for target: CAT_OCUP


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed